## Linear Regression

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import xlrd
from sklearn import linear_model
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

In [2]:
df = pd.read_excel('table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls', header=4, skipfooter=3)

In [3]:
print(df.shape)
df.tail()

(348, 13)


,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
343,Woodbury Town,10685,3,0,nan,0,2,1,541,9,529,3,nan
344,Woodridge Village,829,7,0,nan,0,0,7,17,8,9,0,0.000
345,Woodstock Town,5931,2,0,nan,0,0,2,58,13,45,0,nan
346,Yonkers,199134,1036,6,nan,25,390,615,2368,470,1662,236,10.000
347,Yorktown Town,36643,15,0,nan,0,2,13,334,45,287,2,nan


In [4]:
df.columns

Index(['City', 'Population', 'Violent\ncrime',
       'Murder and\nnonnegligent\nmanslaughter',
       'Rape\n(revised\ndefinition)1', 'Rape\n(legacy\ndefinition)2',
       'Robbery', 'Aggravated\nassault', 'Property\ncrime', 'Burglary',
       'Larceny-\ntheft', 'Motor\nvehicle\ntheft', 'Arson3'],
      dtype='object')

In [5]:
df = df.rename(columns={'Violent\ncrime':'Violent_Crime',
           'Murder and\nnonnegligent\nmanslaughter':'Murder',
           'Rape\n(revised\ndefinition)1':'Rape_Revised',
           'Rape\n(legacy\ndefinition)2':'Rape_Legacy',
           'Aggravated\nassault':'Agg_Assault',
           'Property\ncrime':'Property_Crime',
           'Larceny-\ntheft':'Larceny',
           'Motor\nvehicle\ntheft':'Motor_Vehicle_Theft'
          })

In [6]:
#checking null values
df.isnull().sum()

City                     0
Population               0
Violent_Crime            0
Murder                   0
Rape_Revised           348
Rape_Legacy              0
Robbery                  0
Agg_Assault              0
Property_Crime           0
Burglary                 0
Larceny                  0
Motor_Vehicle_Theft      0
Arson3                 161
dtype: int64

In [7]:
#nulls as percent
df.isnull().sum()/df.shape[0]

City                  0.000
Population            0.000
Violent_Crime         0.000
Murder                0.000
Rape_Revised          1.000
Rape_Legacy           0.000
Robbery               0.000
Agg_Assault           0.000
Property_Crime        0.000
Burglary              0.000
Larceny               0.000
Motor_Vehicle_Theft   0.000
Arson3                0.463
dtype: float64

In [8]:
#100% of Rape_Revised column is null so dropping column
df=df.drop('Rape_Revised', 1)

In [9]:
df[df.Murder.isnull()]

,City,Population,Violent_Crime,Murder,Rape_Legacy,Robbery,Agg_Assault,Property_Crime,Burglary,Larceny,Motor_Vehicle_Theft,Arson3


In [10]:
df.describe()

,Population,Violent_Crime,Murder,Rape_Legacy,Robbery,Agg_Assault,Property_Crime,Burglary,Larceny,Motor_Vehicle_Theft,Arson3
count,348.000,348.000,348.000,348.000,348.000,348.000,348.000,348.000,348.000,348.000,187.000
mean,40037.632,201.595,1.566,5.865,72.902,121.261,792.606,119.684,637.017,35.905,1.872
std,450037.368,2815.269,18.304,60.425,1031.033,1706.132,7659.725,924.949,6346.054,403.424,10.693
min,526.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,3003.000,2.000,0.000,0.000,0.000,1.000,40.500,6.000,31.000,0.000,0.000
50%,7233.500,6.000,0.000,0.000,1.000,4.000,112.500,17.500,94.000,2.000,0.000
75%,18427.500,22.000,0.000,2.000,5.000,14.000,341.000,51.250,287.250,7.000,1.000
max,8396126.000,52384.000,335.000,1112.000,19170.000,31767.000,141971.000,16606.000,117931.000,7434.000,132.000


In [11]:
##not helpful here but

# Print the value counts for categorical columns
#for col in df.columns:
#    if df[col].dtype == 'object':
#        print('\nColumn Name:', col,)
#       print(df[col].value_counts())

## Prepare data to model with multivariable regression according to the specification:

$$  Propertycrime = \alpha + Population + Population^2 + Murder + Robbery $$

 


### Convert Robbery and Murder to Categorical Variables

In [14]:
df.Murder = np.where(df.Murder >= 1, 1, 0)
df.Murder = df.Murder.astype('category')

df.Burglary = np.where(df.Burglary >= 1, 1, 0)
df.Burglary = df.Burglary.astype('category')

### Create new column for  population squared

In [18]:
df['Population2'] = df.Population ** 2